Governed Tags Demo: 
Programatically Creating Governed Tags

In [0]:
import requests
token = dbutils.secrets.get(scope="governed-tags-scope", key="governed-tags-key")

response = requests.get(
    f"https://adb-11214.141512.azuredatabricks.net/api/2.1/unity-catalog/catalogs",
    headers={"Authorization": f"Bearer {token}"}
)
print(response.status_code, response.text)


In [0]:
import requests
import json

# Set your Databricks workspace URL
workspace_url = "https://adb-11214.141512.azuredatabricks.net"  # e.g. https://adb-xxxx.xx.azuredatabricks.net

# Get your access token securely from Databricks secrets
token = dbutils.secrets.get(scope="governed-tags-scope", key="governed-tags-key")

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Define list of governed tags to create
governed_tags = [
    {
        "tag_key": "data_owner",
        "description": "Identifies the team or individual responsible for data quality and lifecycle.",
        "allowed_values": ["Finance", "HR", "Marketing", "Legal", "Operations"]
    },
    {
        "tag_key": "data_classification",
        "description": "Used for regulatory classification such as PII, PCI, HIPAA, GDPR, or None.",
        "allowed_values": ["PII", "PCI", "HIPAA", "GDPR", "None"]
    },
    {
        "tag_key": "data_retention",
        "description": "Specifies retention policy duration.",
        "allowed_values": ["30_days", "1_year", "7_years", "indefinite"]
    },
    {
        "tag_key": "data_source",
        "description": "Indicates the origin of the data.",
        "allowed_values": ["CRM", "ERP", "WebLogs", "IoT"]
    },
    {
        "tag_key": "data_status",
        "description": "Flags data lifecycle stages.",
        "allowed_values": ["Active", "Archived", "Deprecated", "Certified"]
    },
    {
        "tag_key": "project_code",
        "description": "Assigns data assets to specific projects or cost centers.",
        "allowed_values": ["PROJ001", "PROJ002", "PROJ003"]
    },
    {
        "tag_key": "sensitivity_level",
        "description": "Indicates data sensitivity levels.",
        "allowed_values": ["Low", "Medium", "High", "Restricted"]
    },
    {
        "tag_key": "business_unit",
        "description": "Tags assets by organizational unit.",
        "allowed_values": ["Sales", "Operations", "R&D", "Support"]
    },
    {
        "tag_key": "data_quality",
        "description": "Indicates the data quality status.",
        "allowed_values": ["Validated", "Reviewed", "Unverified"]
    },
    {
        "tag_key": "confidentiality_level",
        "description": "Specifies confidentiality governance levels.",
        "allowed_values": ["Internal Use Only", "Confidential", "Top Secret"]
    }
]

# API endpoint
api_url = f"{workspace_url}/api/2.1/unity-catalog/governed-tags"

# Function to create a governed tag
def create_governed_tag(tag):
    response = requests.post(
        api_url,
        headers=headers,
        json=tag
    )
    if response.status_code == 200:
        print(f"Created tag '{tag['tag_key']}' successfully.")
    elif response.status_code == 409:  # Conflict, tag exists
        print(f"Tag '{tag['tag_key']}' already exists.")
    else:
        print(f"Failed to create tag '{tag['tag_key']}': {response.status_code} - {response.text}")

# Iterate over governed tags and create each
for tag in governed_tags:
    create_governed_tag(tag)


Create Catalog, Schema, Table, & Apply Governed Tags to Tables and Columns

In [0]:
%sql
-- Create catalog and schema
CREATE CATALOG IF NOT EXISTS governed_tags_sample;
USE CATALOG governed_tags_sample;

CREATE SCHEMA IF NOT EXISTS silver;
USE SCHEMA silver;

-- Create tables without tags first
CREATE TABLE IF NOT EXISTS customer_public (
  customer_id INT,
  name STRING,
  email STRING
);

CREATE TABLE IF NOT EXISTS employee_confidential (
  employee_id INT,
  name STRING,
  ssn STRING,
  salary DECIMAL(10,2)
);

CREATE TABLE IF NOT EXISTS finance_internal (
  transaction_id INT,
  account_no STRING,
  amount DECIMAL(10,2),
  audit_info STRING
);

-- Apply governed tags on tables
ALTER TABLE governed_tags_sample.silver.customer_public
SET TAGS ('data_owner' = 'Marketing',
          'data_classification' = 'None',
          'data_retention' = '1_year',
          'data_source' = 'CRM',
          'data_status' = 'Active',
          'project_code' = 'PROJ001',
          'sensitivity_level' = 'Low',
          'business_unit' = 'Sales',
          'data_quality' = 'Validated',
          'confidentiality_level' = 'Internal Use Only'
);

ALTER TABLE governed_tags_sample.silver.employee_confidential
SET TAGS ('data_owner' = 'HR',
          'data_classification' = 'PII',
          'data_retention' = '7_years',
          'data_source' = 'ERP',
          'data_status' = 'Active',
          'project_code' = 'PROJ002',
          'sensitivity_level' = 'High',
          'business_unit' = 'Operations',
          'data_quality' = 'Reviewed',
          'confidentiality_level' = 'Confidential'
);

ALTER TABLE governed_tags_sample.silver.finance_internal
SET TAGS ('data_owner' = 'Finance',
          'data_classification' = 'None',
          'data_retention' = '7_years',
          'data_source' = 'WebLogs',
          'data_status' = 'Active',
          'project_code' = 'PROJ003',
          'sensitivity_level' = 'Medium',
          'business_unit' = 'Operations',
          'data_quality' = 'Validated',
          'confidentiality_level' = 'Internal Use Only'
);

-- Apply governed tags on columns
ALTER TABLE governed_tags_sample.silver.customer_public
ALTER COLUMN email SET TAGS ('confidentiality_level' = 'Internal Use Only');

ALTER TABLE governed_tags_sample.silver.employee_confidential
ALTER COLUMN ssn SET TAGS ('sensitivity_level' = 'Restricted');

ALTER TABLE governed_tags_sample.silver.employee_confidential
ALTER COLUMN salary SET TAGS ('confidentiality_level' = 'Confidential');

ALTER TABLE governed_tags_sample.silver.finance_internal
ALTER COLUMN audit_info SET TAGS ('sensitivity_level' = 'Restricted');


Remove Governed Tags from tables and columns

In [0]:
%sql
-- Unset table-level governed tags on customer_public
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public data_owner;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public data_classification;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public data_retention;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public data_source;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public data_status;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public project_code;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public sensitivity_level;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public business_unit;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public data_quality;
UNSET TAG ON TABLE governed_tags_sample.silver.customer_public confidentiality_level;

-- Unset table-level governed tags on employee_confidential
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential data_owner;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential data_classification;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential data_retention;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential data_source;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential data_status;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential project_code;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential sensitivity_level;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential business_unit;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential data_quality;
UNSET TAG ON TABLE governed_tags_sample.silver.employee_confidential confidentiality_level;

-- Unset table-level governed tags on finance_internal
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal data_owner;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal data_classification;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal data_retention;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal data_source;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal data_status;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal project_code;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal sensitivity_level;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal business_unit;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal data_quality;
UNSET TAG ON TABLE governed_tags_sample.silver.finance_internal confidentiality_level;

-- Unset column-level governed tags
UNSET TAG ON COLUMN governed_tags_sample.silver.customer_public.email confidentiality_level;

UNSET TAG ON COLUMN governed_tags_sample.silver.employee_confidential.ssn sensitivity_level;
UNSET TAG ON COLUMN governed_tags_sample.silver.employee_confidential.salary confidentiality_level;

UNSET TAG ON COLUMN governed_tags_sample.silver.finance_internal.audit_info sensitivity_level;


Programatically Delete Governed Tags

In [0]:
import requests

# Your workspace API endpoint (make sure no trailing slash)
workspace_url = "https://adb-11214.141512.azuredatabricks.net"

# Retrieve your token securely from Databricks secrets
token = dbutils.secrets.get(scope="governed-tags-scope", key="governed-tags-key")

headers = {
    "Authorization": f"Bearer {token}"
}

governed_tag_keys = [
    "data_owner",
    "data_classification",
    "data_retention",
    "data_source",
    "data_status",
    "project_code",
    "sensitivity_level",
    "business_unit",
    "data_quality",
    "confidentiality_level"
]

for tag_key in governed_tag_keys:
    response = requests.delete(
        f"{workspace_url}/api/2.1/unity-catalog/governed-tags/{tag_key}",
        headers=headers
    )
    if response.status_code == 200:
        print(f"Deleted governed tag '{tag_key}' successfully.")
    elif response.status_code == 404:
        print(f"Governed tag '{tag_key}' not found.")
    else:
        print(f"Failed to delete governed tag '{tag_key}': {response.status_code} - {response.text}")


Drop Tables & Catalog

In [0]:
%sql
-- Drop all tables by dropping the schema with cascade
DROP SCHEMA IF EXISTS governed_tags_sample.silver CASCADE;

-- Drop the catalog with cascade to remove all schemas and objects completely
DROP CATALOG IF EXISTS governed_tags_sample CASCADE;
